In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (101 kB/s)
Reading package lis

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-24 03:49:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.63MB/s    in 0.2s    

2022-06-24 03:49:32 (5.63 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("census_data_2010s").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://pollution-sight-data.s3.us-west-1.amazonaws.com/census_data_2010s.csv"
spark.sparkContext.addFile(url)
census_2010s_df = spark.read.csv(SparkFiles.get("census_data_2010s.csv"), header=True, inferSchema=True)
census_2010s_df.show()

+-------------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-----------------+---------------+
|               NAME| STNAME|POPESTIMATE2010|POPESTIMATE2011|POPESTIMATE2012|POPESTIMATE2013|POPESTIMATE2014|POPESTIMATE2015|POPESTIMATE2016|POPESTIMATE2017|POPESTIMATE2018|POPESTIMATE2019|POPESTIMATE042020|POPESTIMATE2020|
+-------------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+-----------------+---------------+
|            Alabama|Alabama|        4785514|        4799642|        4816632|        4831586|        4843737|        4854803|        4866824|        4877989|        4891628|        4907965|          4920706|        4921532|
|     Abbeville city|Alabama|           2699|           2694|           2645|           2629|           

In [ ]:
census_2010s_df = census_2010s_df.select(['NAME', 'STNAME', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019', 'POPESTIMATE2020'])
census_2010s_df.show()

+-------------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|               NAME| STNAME|POPESTIMATE2011|POPESTIMATE2012|POPESTIMATE2013|POPESTIMATE2014|POPESTIMATE2015|POPESTIMATE2016|POPESTIMATE2017|POPESTIMATE2018|POPESTIMATE2019|POPESTIMATE2020|
+-------------------+-------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|            Alabama|Alabama|        4799642|        4816632|        4831586|        4843737|        4854803|        4866824|        4877989|        4891628|        4907965|        4921532|
|     Abbeville city|Alabama|           2694|           2645|           2629|           2610|           2602|           2587|           2578|           2565|           2555|           2553|
|    Adamsville city|Alabama|           4474|     

In [ ]:
census_2010s_df.printSchema()

root
 |-- NAME: string (nullable = true)
 |-- STNAME: string (nullable = true)
 |-- POPESTIMATE2011: integer (nullable = true)
 |-- POPESTIMATE2012: integer (nullable = true)
 |-- POPESTIMATE2013: integer (nullable = true)
 |-- POPESTIMATE2014: integer (nullable = true)
 |-- POPESTIMATE2015: integer (nullable = true)
 |-- POPESTIMATE2016: integer (nullable = true)
 |-- POPESTIMATE2017: integer (nullable = true)
 |-- POPESTIMATE2018: integer (nullable = true)
 |-- POPESTIMATE2019: integer (nullable = true)
 |-- POPESTIMATE2020: integer (nullable = true)



In [ ]:
# Connect to AWS RDS instance
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection_string>:5432/<database_name>"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write census_2010s_df to table in RDS
census_2010s_df.write.jdbc(url=jdbc_url, table='census_data_2010s', mode=mode, properties=config)